lab2 class with regs


In [ ]:
class LogisticRegressionGD:
    def __init__(self, threshold=0.5): 
        self.a = np.zeros(2).reshape(1, 2)
        self._estimator_type = "classifier" 
        self.threshold = threshold 
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x @ self.a))    
    def predict_proba(self, x): 
        return self.sigmoid(x)
    def predict(self, x): 
        return (self.predict_proba(x) > self.threshold).astype(int)
    def coefs(self):
        return self.a      
    def LogLikelihood(self, x, Y):
        return (Y * np.log(self.predict_proba(x) + 1e-7) + (1 - Y) * np.log(1 + 1e-7 - self.predict_proba(x))).sum()
    def CrossEntropy(self, x, Y):
        return (-Y * np.log(self.predict_proba(x)) - (1 - Y) * np.log(1 - self.predict_proba(x))).sum()    
    def accuracy(self, x, Y):
        return (self.predict(x) == Y).mean() 
    
    def MSE(self, x, Y):
        return (((Y.to_numpy() - self.predict(x).to_numpy()).T@(Y.to_numpy() - self.predict(x).to_numpy()))/x.shape[0])[0][0]

    def fit(self, x, Y, alpha = 0.01, epsylon = 0.01, max_steps = 10000, Rtype = "LL", intercept = True, reg = "No", lam = 0, rho = 0):
        self.intercept = intercept
        self.x = x.copy()
        self.Y = Y.copy()
        if intercept:
            self.x.insert(0, "intercept", np.ones((self.x.shape[0],1)), allow_duplicates=True)                
        self.a = np.zeros(x.shape[1]).reshape(x.shape[1],1)
        steps, errors = [], []
        step = 0
        for _ in range(max_steps):
            if Rtype == "LL":
                new_error = self.LogLikelihood(x, Y)
                dT_a = x.T @(Y - self.predict_proba(x)) / x.shape[0]
                if reg == "L1":
                    dT_a += lam*(np.sign(self.a))
                elif reg == "L2":
                    dT_a += 2*lam*((self.a))
                elif reg == "ElasticNet":
                    dT_a += rho*lam*(np.sign(self.a)) + 2*(1-rho)*lam*((self.a))  
                self.a += alpha*dT_a
            elif Rtype == "CE":
                new_error = self.CrossEntropy(x, Y)
                #display(new_error)
                dT_a = -x.T @(Y - self.predict_proba(x)) / x.shape[0]
                self.a -= alpha*dT_a
            #new_error = self.MSE(self.x, self.Y)
            step += 1
            steps.append(step)
            errors.append(new_error)